In [15]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None) 
pd.set_option("display.max_colwidth", None)

In [25]:
df = pd.read_csv("../data/raw/boxscore.csv")


df.head()

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,1,3,4,4,1,1,4,5,14,-2,1
1,1,Sacramento Kings,Mitch Richmond,32:00,6,12,1,4,1,1,0,5,5,3,1,2,3,1,14,-12,1
2,1,Sacramento Kings,Olden Polynice,31:34,0,4,0,0,1,4,2,5,7,3,1,0,4,3,1,-12,1
3,1,Sacramento Kings,Mahmoud Abdul-Rauf,29:27,7,13,1,2,2,2,0,2,2,5,1,1,2,2,17,-7,1
4,1,Sacramento Kings,Brian Grant,25:13,3,11,0,0,2,2,1,5,6,0,0,2,1,2,8,-7,1


In [29]:
df = pd.read_csv("../data/raw/games.csv")

df = df[df["seasonStartYear"] >= 2019]

df.head(20)

,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,attendance,notes,startET,datetime,isRegular,game_id
29107,2019,New Orleans Pelicans,122,Toronto Raptors,130,20787.0,NaN,8:00p,2019-10-22,1,29108
29108,2019,Los Angeles Lakers,102,Los Angeles Clippers,112,19068.0,NaN,10:30p,2019-10-22,1,29109
29109,2019,Oklahoma City Thunder,95,Utah Jazz,100,18306.0,NaN,9:00p,2019-10-23,1,29110
29110,2019,Denver Nuggets,108,Portland Trail Blazers,100,19991.0,NaN,10:00p,2019-10-23,1,29111
29111,2019,New York Knicks,111,San Antonio Spurs,120,18354.0,NaN,8:30p,2019-10-23,1,29112
29112,2019,Memphis Grizzlies,101,Miami Heat,120,19600.0,NaN,7:30p,2019-10-23,1,29113
29113,2019,Washington Wizards,100,Dallas Mavericks,108,19816.0,NaN,8:30p,2019-10-23,1,29114
29114,2019,Chicago Bulls,125,Charlotte Hornets,126,15424.0,NaN,7:00p,2019-10-23,1,29115
29115,2019,Detroit Pistons,119,Indiana Pacers,110,17923.0,NaN,7:00p,2019-10-23,1,29116
29116,2019,Cleveland Cavaliers,85,Orlando Magic,94,18846.0,NaN,7:00p,2019-10-23,1,29117


In [24]:
df = pd.read_csv("../data/raw/salaries.csv")

df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary
0,Michael Jordan,1996,"$30,140,000","$52,258,566"
1,Horace Grant,1996,"$14,857,000","$25,759,971"
2,Reggie Miller,1996,"$11,250,000","$19,505,934"
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585"
4,Gary Payton,1996,"$10,212,000","$17,706,187"


In [20]:
df = pd.read_csv("../data/raw/player_info.csv")

df.head()

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"
